# Procesamiento digital de imágenes con aprendizaje automático

El aprendizaje automático (*Machine Learning*) es una disciplina que se enfoca
en el desarrollo de algoritmos que permiten generalizar comportamientos a partir
de muchos ejemplos.
Combina conceptos de matemáticas, estadística y computación; y se puede dividir
en dos grandes categorías: aprendizaje supervisado y no supervisado.


## 1.1. El problema del agrupamiento y la posterización de imágenes

In [ ]:
from sklearn import datasets

datos_iris = datasets.load_iris()
datos_iris.keys()

In [ ]:
X, y = datos_iris["data"], datos_iris["target"]

In [ ]:
import random
indices = random.sample(range(len(X)), k=3)
centroides = X[indices]
centroides

In [ ]:
import math
import numpy as np


def distancia(vec_x, vec_y):
    return math.sqrt(sum((vec_x - vec_y)**2))

def encontar_centroide(X, centroides):
    def distancia_a_centroide(i):
        return distancia(vec_x, centroides[i])
    
    indices = range(len(centroides))
    etiquetas = [None]*len(X)
    for i, vec_x in enumerate(X):
        etiquetas[i] = min(indices, key=distancia_a_centroide)
    
    return etiquetas

def centro_de_masa(X):
    return sum(X)/len(X)

def inercia(X, centroides):
    etiquetas = np.array(encontar_centroide(X, centroides))
    total = 0.0
    for i in range(len(centroides)):
        seleccion = X[etiquetas == i]
        distancias = [distancia(vec_x, centroides[i]) for vec_x in seleccion]
        total += sum(d**2 for d in distancias)
    return total

In [ ]:

indices = random.sample(range(len(X)), k=3)
centroides = X[indices]
print("Inercia inicial:", inercia(X, centroides))

for t in range(15):
    etiquetas = np.array(encontar_centroide(X, centroides))
    for i in range(len(centroides)):
        seleccion = X[etiquetas == i]
        centroides[i] = centro_de_masa(seleccion)
    print(f"Inercia al tiempo {t}: {inercia(X, centroides)}")


In [ ]:
from sklearn.cluster import KMeans

# PASO 1: Definir el modelo
modelo = KMeans(n_clusters=3)
# PASO 2: Realizar entrenamiento
modelo.fit(X)

In [ ]:
modelo.labels_

In [ ]:
datos_iris["target"]

In [ ]:
modelo.inertia_

In [ ]:
import skimage as ski
from materiales.sipi import ejemplo_sipi 

In [ ]:
img = ski.io.imread("sipi/misc/4.2.06.tiff")
img[0, :2]

In [ ]:
alto, ancho, canales = img.shape
X = img.reshape((alto*ancho, canales))

modelo = KMeans(8)
modelo.fit(X)

In [ ]:
modelo.cluster_centers_

In [ ]:
poster = np.array([modelo.cluster_centers_[etiqueta] for etiqueta in modelo.labels_])
poster = poster.reshape((alto, ancho, canales))/255

In [ ]:
ski.io.imshow(poster)

# 2. El algoritmo SIFT para la detección de puntos de interés

Supongamos que tenemos dos fotografías acerca del mismo objeto y son
relativamente similares, como por ejemplo, dos fotografías de una estatua
famosa. Si las dos imágenes son sufiencientemente similares salvo rotación,
acercamiento o cambio de iluminación, entonces sería posible encontrar
correspondencias entre los puntos de interés de ambas imágenes.
Eso es exactamente lo que hace el algoritmo SIFT (*Scale-Invariant Feature
Transform*), que es un algoritmo de visión por computadora que permite
detectar y describir puntos de interés en una imagen.

## 2.1. Aplicación del desenfoque gaussiano

La primera etapa del algoritmo SIFT consiste en aplicar un desenfoque gaussiano
iterativo a la imagen original.
El desenfoque gaussiano es un filtro que se utiliza para atenuar el ruido de
alta frecuencia en una imagen.
Para ello utilizamos la convolución de la imagen original con un kernel
gaussiano, es decir, con una ventana de valores que siguen una distribución
gaussiana.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
from matplotlib import pyplot as plt
from scipy import signal
from skimage import data, filters


def kernel_gaussiano(sigma, tamano = None):
    if tamano is None:
        tamano = int(6*sigma)
    x = np.arange(-tamano // 2, tamano // 2 + 1)
    y = np.arange(-tamano // 2, tamano // 2 + 1)
    xx, yy = np.meshgrid(x, y)
    result = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    # Normalizar para que la suma sea 1
    return result / result.sum()

def mostrar_kernel_gaussiano(sigma):
    kernel = kernel_gaussiano(sigma)
    plt.imshow(kernel, cmap="gray")
    plt.show()

mostrar_kernel_gaussiano(1.618)

In [ ]:
import plotly

# Graficar en 3D la ventana de un kernel gaussiano
kernel = kernel_gaussiano(1.618)
# Usar la función meshgrid para obtener las coordenadas de los puntos
x = np.arange(kernel.shape[0])
y = np.arange(kernel.shape[1])
xx, yy = np.meshgrid(x, y)
# Crear la figura
fig = plotly.graph_objs.Figure(data=[
    plotly.graph_objs.Surface(z=kernel, x=xx, y=yy)
])
# Mostrar la figura en Jupyter
plotly.offline.iplot(fig)

In [ ]:

def convolucion(imagen, kernel):
    return signal.convolve2d(imagen, kernel, mode="same")


def suavizar(imagen, sigma):
    kernel = kernel_gaussiano(sigma)
    if len(imagen.shape) == 2:
        return convolucion(imagen, kernel)
    n_canales = imagen.shape[2]
    imgs = [imagen[:, :, i] for i in range(n_canales)]
    canales = [convolucion(imgs[i], kernel) for i in range(n_canales)]
    return np.stack(canales, axis=2)


def ejemplo_suavizado(img, s, sigma, k=3):
    # Aplicar suavizado con s^1 sigma, s^2 sigma, ..., s^k sigma
    fig, axs = plt.subplots(1, k, figsize=(20, 5))
    axs[0].imshow(img)
    axs[0].axis("off")
    axs[0].set_title("Original")
    for i in range(1, k):
        img_suave = suavizar(img, sigma * s**i)
        axs[i].imshow(img_suave)
        axs[i].axis("off")
        axs[i].set_title(f"Suavizado {i}")

    fig.tight_layout()

img = ski.img_as_float(data.chelsea())
ejemplo_suavizado(img, 1.618, 1.618, 5)

La diferencia de gaussianas, es decir, la resta de dos imágenes desenfocadas
con diferentes valores de sigma, es un método útil para encontrar los bordes
de los objetos en una imagen, ya que los bordes son precisamente las regiones
de la imagen donde los valores de los píxeles cambian abruptamente.

In [ ]:
# Diferencia de gaussianas
def diferencia_gaussianas(img, sigma1, sigma2):
    suave1 = suavizar(img, sigma1)
    suave2 = suavizar(img, sigma2)
    return suave1 - suave2

def ejemplo_diferencia_gaussianas(img, sigma1, sigma2):
    dog = diferencia_gaussianas(img, sigma1, sigma2)
    # Normalizar para mostrar
    dog = (dog - dog.min()) / (dog.max() - dog.min())
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img)
    axs[0].axis("off")
    axs[0].set_title("Original")
    axs[1].imshow(dog, cmap="gray")
    axs[1].axis("off")
    axs[1].set_title("Diferencia de gaussianas")
    fig.tight_layout()

img = ski.img_as_float(data.astronaut())
ejemplo_diferencia_gaussianas(img, 1, 1.001)

In [ ]:
def apilar_dog(img, s, sigma, k):
    # Aplicar suavizado con s^1 sigma, s^2 sigma, ..., s^k sigma
    if len(img.shape) == 3:
        # Convertir a escala de grises
        img = ski.color.rgb2gray(img)
    suves = [suavizar(img, sigma * s**i) for i in range(k)]
    dogs = [suves[i] - suves[i+1] for i in range(k-1)]
    # Apilar diferencias de gaussianas
    return np.stack(dogs, axis=2)

def ejemplo_apilar_dog(img, s, sigma, k):
    dog = apilar_dog(img, s, sigma, k)
    fig, axs = plt.subplots(1, k-1, figsize=(20, 5))
    for i in range(k-1):
        diff = dog[:, :, i]
        # Normalizar
        min_diff = np.min(diff)
        diff = (diff - min_diff)/(np.max(diff) - min_diff)
        axs[i].imshow(diff, cmap="gray")
        axs[i].axis("off")
        axs[i].set_title(f"DOG {i+1}")
    fig.tight_layout()

ejemplo_apilar_dog(img, 1.618, 1.618, 5)

In [ ]:
"Hola"[0:3 + 1]

In [ ]:
import itertools


def encontrar_puntos(dogs):
    # Convertir a valor absoluto el tensor dogs
    dogs = np.abs(dogs)
    alto, ancho, profundo = dogs.shape
    resultado = set()
    for i, j, k in itertools.product(range(alto), range(ancho), range(profundo)):
        vecindad = dogs[
            max(i - 1, 0) : min(i + 2, alto),
            max(j - 1, 0) : min(j + 2, ancho),
            max(k - 1, 0) : min(k + 2, profundo),
        ]
        max_vecindad = np.max(vecindad)
        # Si el píxel dogs[i, j, k] es un máximo o mínimo local, se
        # considera un punto de interés
        if dogs[i, j, k] == max_vecindad:
            resultado.add((i, j))
        
    return resultado

def ejemplo_encontrar_puntos(img, s, sigma, k):
    dogs = apilar_dog(img, s, sigma, k)
    puntos = encontrar_puntos(dogs)
    fig, axs = plt.subplots(1, 1, figsize=(5, 5))
    axs.imshow(img)
    axs.axis("off")
    for i, j in puntos:
        axs.plot(j, i, "ro")
    fig.tight_layout()

ejemplo_encontrar_puntos(img, 1.618, 1.618, 5)

In [ ]:
import matplotlib.pyplot as plt

from skimage import data
from skimage import transform
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, plot_matches, SIFT

img1 = rgb2gray(data.astronaut())
img2 = transform.rotate(img1, 180)
tform = transform.AffineTransform(scale=(1.3, 1.1), rotation=0.5, translation=(0, -200))
img3 = transform.warp(img1, tform)

descriptor_extractor = SIFT()

descriptor_extractor.detect_and_extract(img1)
keypoints1 = descriptor_extractor.keypoints
descriptors1 = descriptor_extractor.descriptors

descriptor_extractor.detect_and_extract(img2)
keypoints2 = descriptor_extractor.keypoints
descriptors2 = descriptor_extractor.descriptors

descriptor_extractor.detect_and_extract(img3)
keypoints3 = descriptor_extractor.keypoints
descriptors3 = descriptor_extractor.descriptors

matches12 = match_descriptors(
    descriptors1, descriptors2, max_ratio=0.6, cross_check=True
)
matches13 = match_descriptors(
    descriptors1, descriptors3, max_ratio=0.6, cross_check=True
)
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(11, 8))

plt.gray()

plot_matches(ax[0, 0], img1, img2, keypoints1, keypoints2, matches12)
ax[0, 0].axis('off')
ax[0, 0].set_title("Original Image vs. Flipped Image\n" "(all keypoints and matches)")

plot_matches(ax[1, 0], img1, img3, keypoints1, keypoints3, matches13)
ax[1, 0].axis('off')
ax[1, 0].set_title(
    "Original Image vs. Transformed Image\n" "(all keypoints and matches)"
)

plot_matches(
    ax[0, 1], img1, img2, keypoints1, keypoints2, matches12[::15], only_matches=True
)
ax[0, 1].axis('off')
ax[0, 1].set_title(
    "Original Image vs. Flipped Image\n" "(subset of matches for visibility)"
)

plot_matches(
    ax[1, 1], img1, img3, keypoints1, keypoints3, matches13[::15], only_matches=True
)
ax[1, 1].axis('off')
ax[1, 1].set_title(
    "Original Image vs. Transformed Image\n" "(subset of matches for visibility)"
)

plt.tight_layout()
plt.show()